In [0]:
""" An Image classifier which classifies the images of flowers. To prevent Overfitting, we use the concept of Data Augmentation to increase 
the number of training examples. It is similar to Cats vs Dogs classifier except here we have more classes unlike in CatsVSDogs where we had
only 2 classes """

try:
  %tensorflow_version 2.x
except:
  pass

# Imports 
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import glob 
import shutil # Helps to move the files
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
# Downloading the dataset and creating a folder to hold it
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
zip_file = tf.keras.utils.get_file(origin=_URL, extract=True, fname="flowers_photos.tgz")
base_dir = os.path.join(os.path.dirname(zip_file),"flower_photos")

In [0]:
# Labels
classes = ['roses', 'daisy', 'dandelion', 'sunflowers', 'tulips']

# Restructuring the dataset such that 80% images are in train folder and 20% in val folder
for cl in classes:
  img_path = os.path.join(base_dir,cl)
  images = glob.glob(img_path + '/*.jpg')
  train, val = images[:round(len(images)*0.8)], images[round(len(images)*0.8):]
  print("{} : {} Images".format(cl,len(images)))
  for t in train:
    if not os.path.exists(os.path.join(base_dir,"train",cl)):
      os.makedirs(os.path.join(base_dir,"train",cl))
    shutil.move(t,os.path.join(base_dir,"train",cl))

  for v in val:
    if not os.path.exists(os.path.join(base_dir,"val",cl)):
      os.makedirs(os.path.join(base_dir,"val",cl))
    shutil.move(v,os.path.join(base_dir,"val",cl))

In [0]:
train_dir = os.path.join(base_dir,'train')
val_dir = os.path.join(base_dir,'val')

In [0]:
batch_size = 100
image_size = 150

In [0]:
def plot_images(images_arr):
  image, axes = plt.subplots(1,5,figsize=(20,20)) # axes is a np array which contain position where we want to plot
  axes = axes.flatten()
  for img,ax in zip(images_arr,axes):
    ax.imshow(img)
  plt.tight_layout()
  plt.show()

image_gen = ImageDataGenerator(rescale=1./255,
                               rotation_range=45,
                               zoom_range=0.5,
                               horizontal_flip=True,
                               width_shift_range=0.15,
                               height_shift_range=0.15)

train_data_gen = image_gen.flow_from_directory(train_dir,
                                         batch_size = batch_size,
                                         target_size = (image_size,image_size),
                                         shuffle=True,
                                         class_mode='sparse')

In [0]:
aug_images = [train_data_gen[0][0][0] for i in range(5)]
plot_images(aug_images)

In [0]:
image_gen_val = ImageDataGenerator(rescale=1./255)

val_data_gen = image_gen_val.flow_from_directory(val_dir,
                             batch_size = batch_size,
                             target_size = (image_size,image_size),
                             class_mode='sparse')

In [0]:
# Model
model = tf.keras.Sequential([
                             tf.keras.layers.Conv2D(16,3,activation='relu'),
                             tf.keras.layers.MaxPooling2D(2,2),

                             tf.keras.layers.Conv2D(32,3,activation='relu'),
                             tf.keras.layers.MaxPooling2D(2,2),

                             tf.keras.layers.Conv2D(64,3,activation='relu'),
                             tf.keras.layers.MaxPooling2D(2,2),
                             
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(512, activation='relu'),
                             tf.keras.layers.Dense(5,activation='softmax')
])


In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
epochs=10
history = model.fit(train_data_gen,
                              validation_data=val_data_gen,
                              epochs=epochs)

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.plot(epochs_range,acc,label="Training Accuracy")
plt.plot(epochs_range,val_acc,label="Validation Accuracy")
plt.legend(loc='lower right')
plt.subplot(1,2,2)
plt.plot(epochs_range,loss,label="Training Loss")
plt.plot(epochs_range,val_loss,label="Validation Loss")
plt.legend(loc='upper right')

# Validation acc turned out to be more than Training Accuracy